In [3]:
import pickle as pkl
import re
from collections import defaultdict

import pandas as pd
import numpy as np
import pyopenms

In [22]:
POOLED_REPORT = '/home/francis/Documents/FomoNet/results/DIA-NN/pooled_report.tsv'
OUT_PEPTIDES = '/home/francis/Documents/FomoNet/results/fomonet_peptides.txt'
FOMONET_ID = '/home/francis/Documents/FomoNet/results/fomonet_id.pkl'
    
with open(POOLED_REPORT, 'r') as f:
    cols = ['Run', 'Protein.Names', 'MS2.Scan', 'Stripped.Sequence',
            'RT.Start', 'RT.Stop', 'Ms1.Profile.Corr', 'Proteotypic']
    pooled = pd.read_csv(f, sep='\t', usecols=cols)

with open(FOMONET_ID, 'rb') as f:
    fomonet_id = pkl.load(f)

In [23]:
# Filter out line with 
pooled = pooled[pooled['Protein.Names'].isin(fomonet_id)]

In [17]:
# QC: does all fomonet_id have peptides in pooled?
pooled['Protein.Names'].unique().shape == np.unique(fomonet_id).shape

True

In [18]:
pooled.columns

Index(['Run', 'Protein.Names', 'Stripped.Sequence', 'RT.Start', 'RT.Stop',
       'Ms1.Profile.Corr', 'MS2.Scan'],
      dtype='object')

In [19]:
res = {prot: defaultdict(lambda: defaultdict(lambda: defaultdict(str)) )
       for prot in pooled['Protein.Names'].unique()}
for _, row in pooled.iterrows():
    run, pg, pep, rt_start, rt_stop, corr, ms_scan = row
    res[pg][pep][run]['corr'] = corr      # MS1 profile correlation
    res[pg][pep][run]['peak_scan'] = ms_scan     # MS2 scan number during precursor peak intensity
    res[pg][pep][run]['rt'] = tuple([rt_start, rt_stop])

In [20]:
res['ENST00000215754-1']

defaultdict(<function __main__.<dictcomp>.<lambda>()>,
            {'LLCGLLAER': defaultdict(<function __main__.<dictcomp>.<lambda>.<locals>.<lambda>()>,
                         {'COVARIS_50_ctr_1': defaultdict(str,
                                      {'corr': 0.994064033,
                                       'peak_scan': 44360,
                                       'rt': (31.97261238, 32.1937294)}),
                          'COVARIS_50_ctr_2': defaultdict(str,
                                      {'corr': 0.9937808514,
                                       'peak_scan': 43910,
                                       'rt': (31.61799812, 31.83340645)}),
                          'COVARIS_50_ctr_3': defaultdict(str,
                                      {'corr': 0.9971487522,
                                       'peak_scan': 43810,
                                       'rt': (31.51408195, 31.72900009)}),
                          'COVARIS_50_ctr_4': defaultdict(str,
           

In [33]:
run, pg, pep, ms_scan = row